In [ ]:
import sys, os
import numpy as np

from pygama.dsp.ProcessingChain import ProcessingChain
from pygama.dsp.processors import *
from pygama.dsp.units import *

from pygama.io import io_base as io

# CONSTANT VARIABLES TO SET UP CONFIG
filename = os.path.expandvars("ADD A FILE HERE!!!")
groupname = "daqdata"
nentries = 1000 # Number of entries in the file to process
start_entry = 0 # First entry to process
verbosity = 2
wfblock = 8

In [ ]:
# Set up file I/O
lh5 = io.LH5Store()
data = lh5.read_object(groupname, filename, n_rows=nentries, start_row=start_entry)

wf_in = data['waveform']['values'].nda
dt = data['waveform']['dt'].nda[0] * unit_parser.parse_unit(data['waveform']['dt'].attrs['units'])

# Set up processing chain
proc = ProcessingChain(block_width=wfblock, clock_unit=dt, verbosity=verbosity)
proc.add_input_buffer("wf", wf_in, dtype='float32')

# Basic Filters
proc.add_processor(mean_stdev, "wf[0:1000]", "bl", "bl_sig")
proc.add_processor(np.subtract, "wf", "bl", "wf_blsub")
proc.add_processor(pole_zero, "wf_blsub", 70*us, "wf_pz")
proc.add_processor(trap_filter, "wf_pz", 10*us, 5*us, "wf_trap")

# Energy calculation
proc.add_processor(np.divide, "wf_trap", 10*us, "trap_norm")
proc.add_processor(np.amax, "trap_norm", 1, "trapE", signature='(n),()->()', types=['fi->f'])

# Current calculation
proc.add_processor(avg_current, "wf_pz", 10, "curr")
proc.add_processor(np.amax, "curr", 1, "curr_amp", signature='(n),()->()', types=['fi->f'])


In [ ]:
# Get the outputs of interest
wf = proc.get_output_buffer("wf_blsub")
trap = proc.get_output_buffer("trap_norm")
trapE = proc.get_output_buffer("trapE")
curr = proc.get_output_buffer("curr")
curr_amp = proc.get_output_buffer("curr_amp")

wfindex = np.arange(trap.shape[-1])

# Execute analysis
proc.execute()


In [ ]:
# Set index counter. Rerun this box to start over the file...
wfnum = 0

In [ ]:
import matplotlib.pyplot as plt

# Draw things!
plt.scatter(wfindex, wf[wfnum])
plt.scatter(wfindex, trap[wfnum])
plt.scatter(wfindex, curr[wfnum])
plt.hlines(trapE[wfnum], wfindex[0], wfindex[-1])
plt.hlines(curr_amp[wfnum], wfindex[0], wfindex[-1])

wfnum+=1